In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

# 2020-02 Parcial

Tenemos un RDD con información de recetas:

(ID_Receta, Nombre, Categoría)

Y otro RDD con los ingredientes de cada receta:

(ID_Receta, Ingrediente, Cantidad_Kg)

Queremos obtener:

* a) Listar todos los ingredientes que aparecen en alguna receta que usa "pollo" indicando en
cuantas recetas el ingrediente y pollo aparecen juntos. El formato de salida es (ingrediente,
cantidad de recetas en que aparece junto con pollo). Por ejemplo, la papa aparece en 10
recetas con pollo, por lo que tendríamos (papa, 10). (50 pts)

* b) Queremos obtener todos los nombres de recetas Mediterráneas que no tengan ni papa ni
pollo entre sus ingredientes.(50 pts)

Resolver los puntos usando la API de RDDs de PySpark.

In [38]:
recetas = [
    (1, 'wok', 'China'),
    (2, 'estofado', 'Italiana'),
    (3, 'tortilla', 'Mediterranea'),
    (4, 'Pollo al horno', 'Mediterranea'),
    (5, 'Ni Pollo ni papa', 'Mediterranea')
]

ingredientes = [
    (1, 'pollo', 0.4),
    (1, 'zanahoria', 0.2),
    (1, 'papa', 0.2),
    (2, 'carne', 1),
    (3, 'papa', 0.5),
    (4, 'pollo', 0.3),
    (4, 'papa', 0.2),
    (4, 'cebolla', 0.1),
    (5, 'carne', 0.1),
    (5, 'sal', 0.01)
]

In [39]:
recetasRDD = sc.parallelize(recetas)
ingredientesRDD = sc.parallelize(ingredientes)

In [40]:
recetasRDD.take(3)

[(1, 'wok', 'China'),
 (2, 'estofado', 'Italiana'),
 (3, 'tortilla', 'Mediterranea')]

In [41]:
ingredientesRDD.take(3)

[(1, 'pollo', 0.4), (1, 'zanahoria', 0.2), (1, 'papa', 0.2)]

## Resolución del parcial
* a) Listar todos los ingredientes que aparecen en alguna receta que usa "pollo" indicando en
cuantas recetas el ingrediente y pollo aparecen juntos. El formato de salida es (ingrediente,
cantidad de recetas en que aparece junto con pollo). Por ejemplo, la papa aparece en 10
recetas con pollo, por lo que tendríamos (papa, 10). (50 pts)

In [42]:
recetas_con_pollo = ingredientesRDD.filter(lambda x : x[1] == 'pollo').map(lambda x: (x[0],x[1]))
recetas_con_pollo.take(3)

[(1, 'pollo'), (4, 'pollo')]

In [43]:
ing_pollo = ingredientesRDD.filter(lambda x: x[1] != 'pollo').map(lambda x: (x[0],x[1])).join(recetas_con_pollo)
ing_pollo.collect()

[(1, ('zanahoria', 'pollo')),
 (1, ('papa', 'pollo')),
 (4, ('papa', 'pollo')),
 (4, ('cebolla', 'pollo'))]

In [44]:
freq_ing_pollo = ing_pollo.map(lambda x: (x[1][0],1)).reduceByKey(lambda x,y: x+y)
freq_ing_pollo.collect()

[('zanahoria', 1), ('papa', 2), ('cebolla', 1)]


* b) Queremos obtener todos los nombres de recetas Mediterráneas que no tengan ni papa ni
pollo entre sus ingredientes.(50 pts)

In [49]:
recetas_mediterraneas = recetasRDD.filter(lambda x: x[2] == 'Mediterranea').map(lambda x: (x[0],x[1]))
recetas_mediterraneas.collect()

[(3, 'tortilla'), (4, 'Pollo al horno'), (5, 'Ni Pollo ni papa')]

In [58]:
recetas_mediterraneas.join(ingredientesRDD.map(lambda x: (x[0],x[1])))\
    .map(lambda x: (x[1][0],1 if x[1][1] in ['pollo','papa'] else 0))\
    .reduceByKey(lambda x,y: x+y).filter(lambda x: x[1] == 0)\
    .map(lambda x: x[0]).collect()

['Ni Pollo ni papa']